# TempoQL Demo

Before running, please `cd` into this repo and run `pip install .` in the virtual environment of your choice. This will install required dependencies so that you can run this notebook.

This example will use the MIMIC-IV-OMOP dataset available [here](https://physionet.org/content/mimic-iv-demo-omop/0.9/). Please download it and set the path to it in the `base_path` variable below.

In [1]:
from tempo_ql import GenericDataset, formats, QueryEngine, FileVariableStore
import duckdb
import numpy as np
import os
from pathlib import Path
import pandas as pd

In [3]:
# make sure this is the correct path to your MIMIC demo data
base_path = Path('tempo_ql/omop/mimic-iv-demo-data-in-the-omop-common-data-model-0.9') / '1_omop_data_csv'
if not base_path.exists():
    raise ValueError("The path does not exist - are you sure you placed the MIMIC demo data in the correct directory?")

# provide your Gemini API key here if you'd like to try the LLM-assisted authoring workflow
gemini_api_key = open("gemini_key.txt").read().strip()

In [4]:
# Add data from the MIMIC-IV OMOP dataset

if not os.path.exists("example.db"):
    local_db = duckdb.connect("example.db")
    for csv_file in base_path.glob('*.csv'):
        table_name = csv_file.stem.replace("2b_", "")
        local_db.execute(f"create table {table_name} as select * from read_csv_auto('{csv_file}', header=true, ignore_errors=true, parallel=false)")
    local_db.close()

# Initialize query engine and variable store
var_store = FileVariableStore('example_cache')
query_engine = QueryEngine(GenericDataset("duckdb:///example.db", formats.omop(id_field='person_id', use_source_concept_ids=True, concept_id_field='concept_id'), verbose=True), 
                           variable_stores=[var_store])

In [5]:
query_engine.dataset.get_scopes()

['Condition',
 'Device',
 'Drug',
 'Measurement',
 'Observation',
 'Observation Period',
 'Person',
 'Procedure',
 'Visit']

In [6]:
# get available concepts
names = query_engine.dataset.list_data_elements(scope='Procedure', return_counts=True)
names.head(20)

Retrieving concepts for table 'procedure_occurrence'


/Users/vsivaram/miniconda3/envs/sepsis-ai/lib/python3.11/site-packages/duckdb_engine/__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


,id,name,count,scope,type
90,2000030019,Pain Assessment Method,3310,Procedure,event
118,2000004014,Arterial line Insertion Date,992,Procedure,event
73,2000004077,20 Gauge Insertion Date,989,Procedure,event
112,2000004015,Arterial line Tubing Change,959,Procedure,event
72,2000004013,Arterial Line Dressing Change,935,Procedure,event
40,2000004009,Multi Lumen Insertion Date,752,Procedure,event
83,2000004010,Multi Lumen Tubing Change,735,Procedure,event
102,2000004008,Multi Lumen Dressing Change,727,Procedure,event
55,2000004004,PICC Line Dressing Change,725,Procedure,event
71,2000004006,Multi Lumen Cap Change,723,Procedure,event


In [ ]:
# Get all visit occurrence IDs
query_engine.get_ids()

## Interactive Python Widget

Looking for inspiration? Try out the following queries. (Note that the MIMIC-IV OMOP sample has some format inconsistencies so some concepts that are available in the traditional MIMIC format may not be queryable.)

**Basic**
* How long are the ICU stays? `duration({Visit})`
* Which concepts are used to label sodium chloride infusions? `{name contains /sodium chloride/i; scope = Drug}`

**Aggregations**
* How often do patients have AFib recorded? `exists {AF (Atrial Fibrillation); scope = Condition} from #now - 24 hours to #now every 24 hours`
* How often are there recent temperature measurements? `count {Temperature Celsius; scope = Measurement} from #now - 4 hours to #now every 4 hours`
* What's the patient's global minimum mean blood pressure? `min {Non Invasive Blood Pressure mean; scope = Measurement} from #mintime to #maxtime`

In [8]:
# Start an interactive widget, using the LLM assistant if API key is provided
widget = query_engine.interactive(api_key=gemini_api_key)
widget

TempoQLWidget(api_status='Configured', ids_length=100, list_names=['Heart Rate', 'Respiratory Rate', 'SR (Sinu…

In [ ]:
# Work in a JSON file to save queries (e.g. for version control)
query_engine.interactive(file_path='test_queries.json', api_key=gemini_api_key)

In [ ]:
# Later, query from your saved file
query_engine.query_from('test_queries.json')

## Running Queries in Code

In [ ]:
# perform one-off queries
visits = query_engine.query("{name contains /insulin/i; scope = Drug}")
visits

In [ ]:
# See what SQL queries were run after your last query_engine.query() call
print(query_engine.dataset._captured_queries)

In [ ]:
# Run future queries only within a random subset if desired
random_sample = np.random.choice(query_engine.get_ids(), size=50)
query_engine.dataset.set_trajectory_ids(random_sample)

In [ ]:
# we can store variables by name...
var_store['SimpleTemperature'] = query_engine.query("union({Temperature Celsius}, ({Temperature Fahrenheit} - 32) * 5 / 9) where #value < 50")

In [ ]:
# ... and then use their results later
query_engine.query("last SimpleTemperature before #now every 24 hours", return_subqueries=True)